In [1]:
# =============================================================================
# EXPERIMENT: XGBoost with Bayesian Hyperparameter Optimization    on preprocessed 4 data
# Description: Use Optuna to find optimal XGBoost parameters
# Expected runtime: 1-2 hours   
# =============================================================================

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
import pickle
from datetime import datetime
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler

c:\Code\BSEcode\Computational Machine Learning\CML_final_project_2025\Classification _HEF\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# =============================================================================
# 1. LOAD PREPROCESSED DATA (no need to preprocess again!)
# =============================================================================
print("Loading preprocessed data...")
X = pd.read_pickle('../../data/processed/X_train_processed.pkl')
y = pd.read_pickle('../../data/processed/y_train.pkl')
X_test = pd.read_pickle('../../data/processed/X_test_processed.pkl')
test_ids = pd.read_pickle('../../data/processed/test_ids.pkl')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Loading preprocessed data...
X shape: (20885, 70)
y shape: (20885,)


In [4]:
# =============================================================================
# 2. DEFINE EXPERIMENT PARAMETERS
# =============================================================================
EXPERIMENT_NAME = "xgboost_optuna"
MODEL_DESCRIPTION = "XGBoost with Bayesian Hyperparameter Optimization on preprocessed 4 data"

In [ ]:
# =============================================================================
# 2. DEFINE OBJECTIVE FUNCTION FOR OPTUNA (With Early Stopping)
# =============================================================================

def objective(trial):
    """
    Optuna objective with Early Stopping.
    """
    
    # 1. Suggest hyperparameters
    # We REMOVE n_estimators from the search space because we let Early Stopping decide it
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'hist',  # Keep 'hist' for speed
        'random_state': 42,
        'n_estimators': 5000,   # Give a high ceiling (budget)
        
        # Learning rate
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        
        # Tree structure (Expanded max_depth slightly for better exploration)
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        
        # Regularization
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        
        # Sampling
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        
        # Handle class imbalance
        'scale_pos_weight': (y == 0).sum() / (y == 1).sum()
    }
    
    # 2. Manual Cross-Validation to enable Early Stopping
    # cross_val_score doesn't support early_stopping well, so we do a loop
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []
    best_ntrees = [] # Track the best iteration for each fold

    for train_idx, val_idx in cv.split(X, y):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        model = xgb.XGBClassifier(**params)
        
        model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            early_stopping_rounds=50, # Stop if no improvement for 50 rounds
            verbose=False
        )
        
        # Get the score from the best iteration
        best_score = model.best_score
        cv_scores.append(best_score)
        best_ntrees.append(model.best_iteration)
    
    # Save the average best_ntrees to use in the final model training
    trial.set_user_attr("n_estimators", int(np.mean(best_ntrees)))
    
    return np.mean(cv_scores)

# =============================================================================
# 3. RUN OPTIMIZATION
# =============================================================================

print("\n" + "="*70)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*70)
print("\nThis will try different hyperparameter combinations intelligently.")
print("Progress will be shown below. This may take 1-2 hours.\n")

# Create study
study = optuna.create_study(
    direction='maximize',  # Maximize ROC-AUC
    sampler=TPESampler(seed=42),
    study_name='xgboost_optimization'
)

# Run optimization
# n_trials: how many different combinations to try
# Higher = better results but takes longer
# 100 trials ≈ 1-2 hours with 5-fold CV
study.optimize(
    objective, 
    n_trials=100,  # Try 100 different combinations
    show_progress_bar=True,
    n_jobs=1  # Run trials sequentially (parallel can cause issues)
)

# =============================================================================
# 4. RESULTS
# =============================================================================

print("\n" + "="*70)
print("OPTIMIZATION COMPLETE")
print("="*70)

print(f"\nBest CV ROC-AUC: {study.best_value:.4f}")
print(f"\nBest hyperparameters:")
for param, value in study.best_params.items():
    print(f"  {param}: {value}")

# Get top 5 trials
#trials_df = study.trials_dataframe()
#top_5 = trials_df.nsmallest(5, 'value')[['number', 'value', 'params']]
#print("\n" + "="*70)
#print("TOP 5 PARAMETER COMBINATIONS")
#print("="*70)
#for idx, row in top_5.iterrows():
#    print(f"\nTrial #{row['number']}: CV ROC-AUC = {row['value']:.4f}")

# =============================================================================
# 5. TRAIN FINAL MODEL WITH BEST PARAMETERS
# =============================================================================

print("\n" + "="*70)
print("TRAINING FINAL MODEL")
print("="*70)

best_params = study.best_params.copy()

# Retrieve the optimal n_estimators found during tuning
optimal_n_estimators = study.best_trial.user_attrs['n_estimators']

best_params.update({
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'random_state': 42,
    'scale_pos_weight': (y == 0).sum() / (y == 1).sum(),
    'n_estimators': optimal_n_estimators # <--- USE THE CALCULATED VALUE
})

# Train on full dataset
model = xgb.XGBClassifier(**best_params)
model.fit(X, y, verbose=False)

# =============================================================================
# 6. FEATURE IMPORTANCE
# =============================================================================

print("\n" + "="*70)
print("FEATURE IMPORTANCE (Top 20)")
print("="*70)

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n" + feature_importance.head(20).to_string(index=False))

# =============================================================================
# 7. GENERATE PREDICTIONS
# =============================================================================

[I 2025-11-23 00:15:48,910] A new study created in memory with name: xgboost_optimization



STARTING BAYESIAN OPTIMIZATION

This will try different hyperparameter combinations intelligently.
Progress will be shown below. This may take 1-2 hours.



Best trial: 0. Best value: 0.872781:   1%|          | 1/100 [00:03<06:21,  3.86s/it]

[I 2025-11-23 00:15:52,765] Trial 0 finished with value: 0.8727811474297491 and parameters: {'learning_rate': 0.03574712922600244, 'n_estimators': 481, 'max_depth': 8, 'min_child_weight': 30, 'max_leaves': 15, 'gamma': 0.7799726016810132, 'reg_alpha': 0.5808361216819946, 'reg_lambda': 8.661761457749352, 'subsample': 0.8005575058716043, 'colsample_bytree': 0.8540362888980227, 'colsample_bylevel': 0.5102922471479012}. Best is trial 0 with value: 0.8727811474297491.


Best trial: 0. Best value: 0.872781:   2%|▏         | 2/100 [00:07<05:38,  3.46s/it]

...

[I 2025-11-23 00:19:24,348] Trial 97 finished with value: 0.8726638421392805 and parameters: {'learning_rate': 0.015508972468119046, 'n_estimators': 487, 'max_depth': 9, 'min_child_weight': 34, 'max_leaves': 44, 'gamma': 3.2196323979090136, 'reg_alpha': 4.412974950697285, 'reg_lambda': 7.1192218203080255, 'subsample': 0.7227509061526945, 'colsample_bytree': 0.5268407233195035, 'colsample_bylevel': 0.7529553193639256}. Best is trial 82 with value: 0.8734707185294056.


Best trial: 82. Best value: 0.873471:  99%|█████████▉| 99/100 [03:38<00:02,  2.63s/it]

[I 2025-11-23 00:19:27,021] Trial 98 finished with value: 0.8728473904173077 and parameters: {'learning_rate': 0.018654730206395093, 'n_estimators': 474, 'max_depth': 8, 'min_child_weight': 34, 'max_leaves': 47, 'gamma': 3.3230851809106614, 'reg_alpha': 3.4202470604752375, 'reg_lambda': 4.9102240398878845, 'subsample': 0.6505128107704424, 'colsample_bytree': 0.5378338780922923, 'colsample_bylevel': 0.6446455146802298}. Best is trial 82 with value: 0.8734707185294056.


Best trial: 82. Best value: 0.873471: 100%|██████████| 100/100 [03:40<00:00,  2.21s/it]


[I 2025-11-23 00:19:29,538] Trial 99 finished with value: 0.8715634725126102 and parameters: {'learning_rate': 0.011363137202300597, 'n_estimators': 444, 'max_depth': 8, 'min_child_weight': 36, 'max_leaves': 38, 'gamma': 2.6509546228829706, 'reg_alpha': 5.407972932933622, 'reg_lambda': 5.310967662327653, 'subsample': 0.6990667006503917, 'colsample_bytree': 0.5732331225225323, 'colsample_bylevel': 0.6257775420632647}. Best is trial 82 with value: 0.8734707185294056.

OPTIMIZATION COMPLETE

Best CV ROC-AUC: 0.8735

Best hyperparameters:
  learning_rate: 0.014713408960258587
  n_estimators: 489
  max_depth: 8
  min_child_weight: 29
  max_leaves: 37
  gamma: 3.5374564635517607
  reg_alpha: 3.673215293829035
  reg_lambda: 4.94160472036624
  subsample: 0.7379806291764146
  colsample_bytree: 0.5118272238342249
  colsample_bylevel: 0.7033436689294454

TRAINING FINAL MODEL

Training ROC-AUC: 0.9387
CV ROC-AUC: 0.8735
Gap (Train - CV): 0.0653
✓ Good! Acceptable overfitting

FEATURE IMPORTANCE (T

In [6]:
# =============================================================================
# 4. GENERATE PREDICTIONS
# =============================================================================
test_proba = model.predict_proba(X_test)[:, 1]


print(f"\nPrediction statistics:")
print(f"  Min: {test_proba.min():.4f}")
print(f"  Max: {test_proba.max():.4f}")
print(f"  Mean: {test_proba.mean():.4f}")
print(f"  Median: {np.median(test_proba):.4f}")


Prediction statistics:
  Min: 0.0034
  Max: 0.9906
  Mean: 0.3287
  Median: 0.2646


In [9]:
# =============================================================================
# 5. CREATE SUBMISSION FILE
# =============================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
submission_filename = f"../../outputs/predictions/XG_boost/{EXPERIMENT_NAME}_{timestamp}.csv"

submission = pd.DataFrame({
    'icustay_id': test_ids,
    'prediction': test_proba
})
submission.to_csv(submission_filename, index=False)
print(f"\n✓ Submission saved: {submission_filename}")


✓ Submission saved: ../../outputs/predictions/XG_boost/xgboost_optuna_20251123_0020.csv


In [10]:
# =============================================================================
# 6. LOG EXPERIMENT (CRITICAL FOR TRACKING!)
# =============================================================================
experiment_log = {
    'timestamp': timestamp,
    'experiment_name': experiment_name,
    'model_type': 'XGBoost (Bayesian Optimization)',
    'n_trials': 100,
    'cv_roc_auc_mean': study.best_value,
    'train_roc_auc': train_auc,
    'overfitting_gap': gap,
    'n_features': X.shape[1],
    'best_params': str(best_params),
    'kaggle_public_score': None,
    'kaggle_private_score': None,
    'submission_file': submission_file
}


# Append to experiment log
log_file = '../experiment_log.csv'
log_df = pd.DataFrame([experiment_log])

import os
if os.path.exists(log_file):
    log_df.to_csv(log_file, mode='a', header=False, index=False)
else:
    log_df.to_csv(log_file, index=False)

print(f"✓ Experiment logged to {log_file}")

NameError: name 'experiment_name' is not defined

In [11]:
# =============================================================================
# 7. SAVE MODEL (OPTIONAL)
# =============================================================================
import pickle


model_filename = f"../../outputs/models/{EXPERIMENT_NAME}_{timestamp}.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved: {model_filename}")

print(f"\n{'='*70}")
print("EXPERIMENT COMPLETE!")
print(f"{'='*70}")
print(f"\nNext steps:")
print(f"1. Upload {submission_filename} to Kaggle")
print(f"2. Record Kaggle score in {log_file}")
print(f"3. Copy this notebook and modify for next experiment")

✓ Model saved: ../../outputs/models/xgboost_optuna_20251123_0020.pkl

EXPERIMENT COMPLETE!

Next steps:
1. Upload ../../outputs/predictions/XG_boost/xgboost_optuna_20251123_0020.csv to Kaggle


NameError: name 'log_file' is not defined